# Calculate and add Basic Path Loss to GDF 

In [2]:
def calculate_Pathloss(gdf,para) :    
    
    # =================================================
    for index, row in gdf.iterrows():
        
        if row['SAT_EL'] >= 10 :  # 3GPP consider the SAT elevation angle > 10 
        
            chan = Channel()
            chan.Altitude         = row['SAT_h']
            chan.Environment      = para.Environment
            chan.State            = row['State']
            chan.ElevationAngle   = row['SAT_EL']
            chan.CarrierFrequency = para.Fc

            if ( 2*10e9 < chan.CarrierFrequency and chan.CarrierFrequency <  4*10e9) : chan.Band = 'S'
            if (26*10e9 < chan.CarrierFrequency and chan.CarrierFrequency < 40*10e9) : chan.Band = 'Ka'

            # Free Space Path Loss
            # =================================================
            fspl, Slant_range = FSPL(chan)
            gdf.at[index, 'FSPL'+' '+str(chan.Environment)+' '+str(chan.CarrierFrequency)] = fspl
            gdf.at[index, 'Slant_range'+' '+str(chan.Environment)+' '+str(chan.CarrierFrequency)] = Slant_range

            # Cluster Loss
            # =================================================
            cl = CL(chan)
            gdf.at[index, 'Cluster Loss'+' '+str(chan.Environment)+' '+str(chan.CarrierFrequency)] = cl

            # Shadow Fading 
            # =================================================
            sf,sigma = SF(chan)
            gdf.at[index, 'SF_sigma'+' '+str(chan.Environment)+' '+str(chan.CarrierFrequency)] = sigma

            # Basic Path Loss (without SF)
            # =================================================
            gdf['PL_b'+' '+str(chan.Environment)+' '+str(chan.CarrierFrequency)] = gdf['FSPL'+' '+str(chan.Environment)+' '+str(chan.CarrierFrequency)] + gdf['Cluster Loss'+' '+str(chan.Environment)+' '+str(chan.CarrierFrequency)]
        
    return gdf

# Calculate Free Space Path Loss

In [4]:
def FSPL(channel) :
    
    # d    : separation distance in meter
    # fc   : frequency in GHz
    # FSPL : free space path loss in dB
    
    fc = channel.CarrierFrequency / (10**9)
    d = Slant_range(channel)
    
    FSPL = 32.45 + 20*math.log10(fc) + 20* math.log10(d)
    #print('fc PL : ' + str(20*math.log10(fc)) + ' dB ')
    #print('d  PL : ' + str(20*math.log10(d) ) + ' dB ')
    #print('FSPL is ' + str(FSPL) + ' dB ')
    
    return (FSPL, d)

In [5]:
def Slant_range(channel) :
    
    # h0   : atellite/HAPS altitude
    # fc   : atellite/HAPS elevation angle
    # RE   : Earth radius
    
    h0    = channel.Altitude * (10**3) # meter
    alpha = channel.ElevationAngle
    Re    = 6371 * (10**3) # meter
    
    
    Slant_range = math.sqrt(Re**2 * math.sin(alpha)**2 + h0**2 + 2*h0*Re) - Re*math.sin(alpha)
    
    #print('Slant range is ' + str(Slant_range) + ' (meter)')
    return Slant_range

In [6]:
# 驗證網站 : https://www.pasternack.com/t-calculator-fspl.aspx

# Calculate Cluster Loss

In [7]:
def CL(channel):

    elevation   = channel.ElevationAngle
    band        = channel.Band  
    environment = channel.Environment
    
    
    # Cluster Loss in dB
    
    if band == 'S' : band_env = 0
    if band == 'Ka': band_env = 1
    elv = round(elevation/10)-1
    
    if environment == 'DenseUrban' :
        Cluster_Loss_dict = [
            [34.3, 44.3],
            [30.9, 39.9],
            [29.0, 37.5],
            [27.7, 35.8],
            [26.8, 34.6],
            [26.2, 33.8],
            [25.8, 33.3],
            [25.5, 33.0],
            [25.5, 32.9]
        ]
    
    if environment == 'Urban' :       
        Cluster_Loss_dict = [
            [34.3, 44.3],
            [30.9, 39.9],
            [29.0, 37.5],
            [27.7, 35.8],
            [26.8, 34.6],
            [26.2, 33.8],
            [25.8, 33.3],
            [25.5, 33.0],
            [25.5 ,32.9]
        ]
    if environment == 'SuburbanAndRural' :
        Cluster_Loss_dict = [
            [19.52, 29.5],
            [18.17, 24.6],
            [18.42, 21.9],
            [18.28, 20.0],
            [18.63, 18.7],
            [17.68, 17.8],
            [16.50, 17.2],
            [16.30, 16.9],
            [16.30 ,16.8]
        ]

    CL = Cluster_Loss_dict[elv][band_env]
    #print('Cluseter Loss is ' + str(CL) + 'dB')
    return CL

# Calculate Shadow Fading

In [ ]:
def SF(channel):
    
    # modeled by a log-normal distribution
    # expressed in decibel unit
    # zero-mean normal distribution 
    # standard deviation : sigma_SF**2
    
    # REF : https://numpy.org/doc/stable/reference/random/generated/numpy.random.lognormal.html
    # use numpy lognormal function
    
    elv   = channel.ElevationAngle
    state = channel.State
    band  = channel.Band
    env   = channel.Environment
    
    sigma = Shadow_Fading_sigma(elv,state,band,env)
    #print('SF sigma is ' + str(sigma))
    
    mu = 0 
    size = 10000
    Shadow_Fading = np.random.lognormal(mu, sigma, size)
    """
    plt.figure()
    #plt.hist(Shadow_Fading, density=True, edgecolor='black',bins = 100)
    mybin = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    count, bins, ignored = plt.hist(Shadow_Fading, density=True, edgecolor='black',bins = mybin)
    
    #print('max:')
    #print(count.max())
    x = np.linspace(min(bins), max(bins), 10000)
    pdf = (np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2))
       / (x * sigma * np.sqrt(2 * np.pi)))
    plt.plot(x, pdf, linewidth=2, color='r')
    
    plt.xlim([0,mybin[-1]])
    plt.ylim([0,count.max()])
    plt.title('Shadow Fading Hist,mu = ' + str(mu) + ', size = ' + str(size) + ', sigma = ' + str(sigma))
    plt.xlabel('dB')
    plt.ylabel('probility')
    plt.show()
    """
    return Shadow_Fading, sigma

In [6]:
def Shadow_Fading_sigma(elevation,state,band,environment) :
    
    # UE in a particular scenario should 
    # take the values corresponding to the reference angle 
    # nearest to its elevation angle a.
    
    # Shadow_Fading_sigma in dB
    
    elv = round(elevation/10)-1
    if state == 'LOS'  : st = 0
    if state == 'NLOS' : st = 1

    #=============================================================================
    if environment == 'DenseUrban' :
        
        if band == 'S' :
            SF_dict = [
                [3.5,15.5],
                [3.4,13.9],
                [2.9,12.4],
                [3.0,11.7],
                [3.1,10.6],
                [2.7,10.5],
                [2.5,10.1],
                [2.3,9.2],
                [1.2,9.2]
            ]
            SF_sigma = SF_dict[elv][st]
        
        if band == 'Ka' :
            SF_dict = [
                [2.9, 17.1],
                [2.4, 17.1],
                [2.7, 15.6],
                [2.4, 14.6],
                [2.4, 14.2],
                [2.7, 12.6],
                [2.6, 12.1],
                [2.8, 12.3],
                [0.6, 12.3]
            ]
            SF_sigma = SF_dict[elv][st]
    #=============================================================================
    if environment == 'Urban' :        

        if band == 'S' :
            SF_dict = [
                [4,6],
                [4,6],
                [4,6],
                [4,6],
                [4,6],
                [4,6],
                [4,6],
                [4,6],
                [4,6]
            ]
            SF_sigma = SF_dict[elv][st]
        
        if band == 'Ka' :
            SF_dict = [
                [4,6],
                [4,6],
                [4,6],
                [4,6],
                [4,6],
                [4,6],
                [4,6],
                [4,6],
                [4,6]
            ]
            SF_sigma = SF_dict[elv][st]
            
    #=============================================================================
    if environment == 'SuburbanAndRural' :
    
        if band == 'S' :
            SF_dict = [
                [1.79, 8.93],
                [1.14, 9.08],
                [1.14, 8.78],
                [0.92, 10.25],
                [1.42, 10.56],
                [1.56, 10.74],
                [0.85, 10.17],
                [0.72, 11.52],
                [0.72, 11.52]
            ]
            SF_sigma = SF_dict[elv][st]
        
        if band == 'Ka' :
            SF_dict = [
                [1.9, 10.7],
                [1.6, 10.0],
                [1.9, 11.2],
                [2.3, 11.6],
                [2.7, 11.8],
                [3.1, 10.8],
                [3.0, 10.8],
                [3.6, 10.8],
                [0.4, 10.8]
            ]
            SF_sigma = SF_dict[elv][st]
    
    # print('SF_sigma' + SF_sigma)
    return SF_sigma